In [ ]:
# Mount path for Google Colab
from google.colab import drive
drive.mount('/content/drive/')

root_path = '/content/drive/My Drive/comp90042-project1/project-data/'

Mounted at /content/drive/


In [ ]:
# Installing libraries
!pip install torch torchvision transformers nltk

     |████████████████████████████████| 2.1MB 4.3MB/s 
     |████████████████████████████████| 3.3MB 29.0MB/s 
     |████████████████████████████████| 901kB 34.0MB/s 


In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 144.6MB 89kB/s 
     |████████████████████████████████| 61kB 2.8MB/s 
ERROR: earthengine-api 0.1.260 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [ ]:
import torch

#[FOR TPU]
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu

#[FOR TPU] Checking device
t = torch.randn(2, 2, device=xm.xla_device())
print(t.device)
print(t)

xla:1
tensor([[ 0.1028, -1.4783],
        [-0.4271,  1.3415]], device='xla:1')


In [ ]:
# Inspect train data using pandas
import pandas as pd

train_df = pd.read_json(root_path + 'train.data.jsonl', lines=True)
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345
0,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...",None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...",None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated

In [ ]:
# Build a preprocessing function for PyTorch DataLoader
import json
import pandas as pd
import re

import nltk
from nltk.corpus import wordnet
from nltk.corpus import brown
from nltk.corpus import reuters
nltk.download('words')
nltk.download('brown')
nltk.download('wordnet')
nltk.download('reuters')

# Uncomment this if doing for the first time
!unzip /root/nltk_data/corpora/reuters.zip -d /root/nltk_data/corpora

import torch
from torch.utils.data import Dataset

from sklearn import preprocessing

from transformers import AutoTokenizer

class PreprocessDataset(Dataset):

    def __init__(self, datafile, labelfile, maxlen):
        # Define constants
        self.REPLY_ID = 'in_reply_to_status_id_str'
        self.TEXT = 'text'

        # Initialise lemmatizer
        self.lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

        # Initialise words corpus form nltk
        
        words = set(nltk.corpus.words.words()) # a list of words provided by NLTK
        brown_words = set(brown.words())
        for w in brown_words:
            words.add(w)
        reuters_words = set(reuters.words())
        for w in reuters_words:
            words.add(w)
        self.words = set([ word.lower() for word in words ]) # lowercase all the words for better matching

        # Adding word that not exists in corpus
        self.words.add('wiki')
        self.words.add('hebdo')

        # Initialise label encoder
        self.le = preprocessing.LabelEncoder()

        # Store train dataset in pandas dataframe
        self.train_df = pd.read_json(datafile, lines=True)

        # Store labels as a list encoded with 0 and 1
        # If labels == None, then preprocessing is done for test dataset
        if labelfile != None:
            with open(labelfile, 'r') as f:
                load_labels = json.load(f)
                self.labels = self.le.fit_transform(list(load_labels.values())).tolist()
        else:
            self.labels = None
        
        # Initialise BERT tokenizer using AutoTokenizer
        self.bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

        # The max length of the tokens
        self.maxlen = maxlen
    
    def lemmatize(self, word):
        lemma = self.lemmatizer.lemmatize(word, 'v')
        if lemma == word:
            lemma = self.lemmatizer.lemmatize(word, 'n')
        return lemma

    def max_match(self, hashtag):
        hashtag_list = []
        i = 1
        len_hashtag = len(hashtag)

        while i < len_hashtag:
            max_word = ''
            for j in range(i, len_hashtag):
                # If non-alphabet, make it as a single word
                if hashtag[i].isnumeric():
                    max_word = hashtag[i]
                    break
                temp_word = hashtag[i:j+1]
                # Lemmatise the word first
                lemma_word = self.lemmatize(temp_word)
                # Then check the lemmaword to corpus
                if lemma_word != temp_word or lemma_word in self.words:
                    # Also check if the temp_word is greater than max_word
                    # Add that to max_word
                    if lemma_word in self.words and len(temp_word) > len(max_word):
                        max_word = temp_word
            
            # If there is no word detected, tokenise the single word
            if len(max_word) == 0:
                max_word = hashtag[i]
            # Move i with respect to max_word
            i += len(max_word)
            hashtag_list.append(max_word)

        return hashtag_list
    
    def max_match_rev(self, hashtag):
        
        hashtag_list = []
        # Append the hashtag since it already tokenised
        i = len(hashtag) - 1
        last_index = len(hashtag) - 1
        
        # Iterate until last_index goes to 0 since it is reversed
        while last_index > 0:
            max_word = ''
            for j in range(last_index, 0, -1):
                # If non-alphabet, make it as a single word
                if hashtag[last_index].isnumeric():
                    max_word = hashtag[last_index]
                    break
                temp_word = hashtag[j:last_index + 1]
                # Lemmatise the word first
                lemma_word = self.lemmatize(temp_word)
                # Then check the lemmaword to corpus
                if lemma_word != temp_word or lemma_word in self.words:
                    # Also check if the temp_word is greater than max_word
                    # Add that to max_word
                    if lemma_word in self.words and len(temp_word) > len(max_word):
                        max_word = temp_word
            
            # If there is no word detected, tokenise the single word
            if len(max_word) == 0:
                max_word = hashtag[last_index]
            # Move last_index with respect to max_word
            last_index -= len(max_word)
            # Insert the token into index 0
            hashtag_list.insert(0, max_word)
        
        return hashtag_list
    
    def simplify_tweet(self, tweet):
        tweet_text = tweet.lower()
        # Remove links
        # Link: https://www.codegrepper.com/code-examples/python/remove+urls+from+text+python
        tweet_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', tweet_text, flags=re.MULTILINE)
        # Remove twitter handle
        # Link: https://stackoverflow.com/questions/50830214/remove-usernames-from-twitter-data-using-python
        tweet_text = re.sub('@[^\s]+', '', tweet_text, flags=re.MULTILINE)
        # Use max_match function for hashtag(s), then append the result back to words
        tweet_text_list = tweet_text.split()
        for word in tweet_text_list:
            if word.startswith("#") and len(word) > 1:
                hashtag_index = tweet_text_list.index(word)
                hashtag_list = self.max_match_rev(word)
                tweet_text_list.remove(word)
                for tag in reversed(hashtag_list):
                    tweet_text_list.insert(hashtag_index, tag)
        separator = ' '
        tweet_text = separator.join(tweet_text_list)

        return tweet_text
    
    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, index):
        # Start preprocessing train_df
        row_tweet = self.train_df.iloc[index]
        source = ''
        replies = ''

        for tweet in row_tweet:
            if tweet == None:
                break
            if tweet[self.REPLY_ID] == None:
                source = self.simplify_tweet(tweet[self.TEXT])
            else:
                replies += self.simplify_tweet(tweet[self.TEXT]) + ' '
        
        bert_tokens = self.bert_tokenizer(source,
                                          replies,
                                          padding='max_length',
                                          truncation=True,
                                          max_length=self.maxlen)
        
        # Example output of bert_tokens
        # {'input_ids': [101, 138, 18696, 155, 1942, 3190, 1144, 1572, 13745, 1104, 159, 9664, 2107, 102],
        # 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        # 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
        
        if self.labels != None:
            label = self.labels[index]
        else:
            label = None

        tokens_ids_t = torch.tensor(bert_tokens['input_ids'])
        attn_mask_t = torch.tensor(bert_tokens['attention_mask'])
        seg_ids_t = torch.tensor(bert_tokens['token_type_ids'])

        if label == None:
            return tokens_ids_t, attn_mask_t, seg_ids_t


        return tokens_ids_t, attn_mask_t, seg_ids_t, label


In [ ]:
from torch.utils.data import DataLoader

MAX_LEN = 512
BATCH_SIZE = 8
SHUFFLE = True

# [FOR GPU]
NUM_WORKERS = 2

# [FOR TPU]
# NUM_WORKERS = 5

# creating instances of training and development set
train_set = PreprocessDataset(datafile=root_path + 'train.data.jsonl',
                              labelfile=root_path + 'train.label.json',
                              maxlen=MAX_LEN)
dev_set = PreprocessDataset(datafile=root_path + 'dev.data.jsonl',
                            labelfile=root_path + 'dev.label.json',
                            maxlen=MAX_LEN)

# creating instances of training and development dataloaders
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=SHUFFLE, num_workers=NUM_WORKERS)
dev_loader = DataLoader(dev_set, batch_size=BATCH_SIZE, shuffle=SHUFFLE, num_workers=NUM_WORKERS)

print('Done preprocessing')


Done preprocessing


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class RumourClassifier(nn.Module):

    def __init__(self):
        super(RumourClassifier, self).__init__()

        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')

        # classification layer
        self.cls_layer = nn.Linear(768, 1)
    
    def forward(self, seq, attn_masks, seg_ids):
        outputs = self.bert_layer(seq,
                                  attention_mask=attn_masks,
                                  token_type_ids=seg_ids)
        cont_reps = outputs.last_hidden_state

        cls_rep = cont_reps[:, 0]

        logits = self.cls_layer(cls_rep)

        return logits

In [ ]:
# [FOR GPU]
gpu = 0

print("Creating the rumour classifier, initialised with pretrained BERT-BASE parameters...")
net = RumourClassifier()
net.cuda(gpu) # Enable gpu support for the model
print("Done creating the rumour classifier.")

# [FOR TPU]
# Only instantiate model weights once in memory.
# print("Creating the rumour classifier, initialised with pretrained BERT-BASE parameters...")
# WRAPPED_MODEL = xmp.MpModelWrapper(RumourClassifier())

# device = xm.xla_device() 
# net = WRAPPED_MODEL.to(device) # Enable TPU support for the model
# print("Done creating the rumour classifier.")

Creating the rumour classifier, initialised with pretrained BERT-BASE parameters...



Done creating the rumour classifier.


In [ ]:
import torch.nn as nn
import torch.optim as optim

lr = 2e-5
# [FOR TPU] need to scale learning rate to world size.
# lr = lr * xm.xrt_world_size() # remove this if you want to run with GPU

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = lr)

In [ ]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    # [FOR TPU] Using ParalellLoader
    # dataloader = pl.ParallelLoader(dataloader, [device])
    # dataloader = dataloader.per_device_loader(device)

    with torch.no_grad():
        for seq, attn_masks, seg_ids, labels in dataloader:
            # [FOR GPU] Converting these to cuda tensors
            seq, attn_masks, seg_ids, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks, seg_ids)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

In [ ]:
import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):

        # [FOR TPU] Using ParalellLoader
        # train_loader = pl.ParallelLoader(train_loader, [device])
        # train_loader = train_loader.per_device_loader(device)
        # dev_loader = pl.ParallelLoader(dev_loader, [device])
        # dev_loader = dev_loader.per_device_loader(device)
        
        for it, (seq, attn_masks, seg_ids, labels) in enumerate(train_loader):
            # clear gradients
            opti.zero_grad()

            # [FOR GPU] converting these to cuda tensors
            seq, attn_masks, seg_ids, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks, seg_ids)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            # [FOR GPU] Optimization step
            opti.step()

            # [FOR TPU] Optimization step
            # xm.optimizer_step(opti)
            
            if it % 100 == 0:
                #Please remove [xla:{}] and xm.get_ordinal() if you want to run with GPU                
                acc = get_accuracy_from_logits(logits, labels)
                # print("[xla:{}] Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(xm.get_ordinal(), it, ep, loss.item(), acc, (time.time()-st)))
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        print("Saving model..")
        torch.save(net.state_dict(), root_path + 'ica_rumor_replies_simplified{}.dat'.format(ep))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}".format(best_acc, dev_acc))
            best_acc = dev_acc

In [ ]:
num_epoch = 2

# [FOR TPU]
# train(net, criterion, opti, train_loader, dev_loader, num_epoch, device)

# [FOR GPU]
train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

Iteration 0 of epoch 0 complete. Loss: 0.6747140884399414; Accuracy: 0.5; Time taken (s): 4.857764005661011
Iteration 100 of epoch 0 complete. Loss: 1.0872360467910767; Accuracy: 0.5; Time taken (s): 148.50611472129822
Iteration 200 of epoch 0 complete. Loss: 0.5765824317932129; Accuracy: 0.75; Time taken (s): 148.50080108642578
Iteration 300 of epoch 0 complete. Loss: 0.2998881936073303; Accuracy: 0.875; Time taken (s): 148.55505466461182
Iteration 400 of epoch 0 complete. Loss: 0.7204743027687073; Accuracy: 0.75; Time taken (s): 148.4182047843933
Iteration 500 of epoch 0 complete. Loss: 0.22614458203315735; Accuracy: 1.0; Time taken (s): 148.40681266784668
Epoch 0 complete! Development Accuracy: 0.8732876777648926; Development Loss: 0.27410470531003117
Saving model..
Best development accuracy improved from 0 to 0.8732876777648926
Iteration 0 of epoch 1 complete. Loss: 0.2706157863140106; Accuracy: 0.875; Time taken (s): 167.75262236595154
Iteration 100 of epoch 1 complete. Loss: 0.06

In [ ]:
net.load_state_dict(torch.load(root_path + '.dat'))

In [ ]:
import json
from torch.utils.data import DataLoader

def generate_prediction(dataset, maxlen):

    REPLY_ID = 'in_reply_to_status_id_str'
    ID = 'id_str'

    predictions = dict()
    id_list = []
    preds = []

    test_df = pd.read_json(dataset, lines=True)

    test_set = PreprocessDataset(datafile=dataset,
                                 labelfile=None,
                                 maxlen=maxlen)
    
    test_loader = DataLoader(test_set)

    with torch.no_grad():
        for seq, attn_masks, seg_ids in test_loader:
            seq, attn_masks, seg_ids = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu)
            logits = net(seq, attn_masks, seg_ids)
            probs = torch.sigmoid(logits)
            soft_prob = (probs > 0.5).long()
            decoded_pred = 'non-rumour' if soft_prob.squeeze().item() == 0 else 'rumour'
            preds.append(decoded_pred)
    
    for index, row in test_df.iterrows():
        for tweet in row:
            if tweet[REPLY_ID] == None:
                id_list.append(tweet[ID])
                break
    
    for id, pred in zip(id_list, preds):
        predictions[id] = pred
    
    with open(root_path + 'test-output.json', 'w') as outfile:
        json.dump(predictions, outfile)
    
    print('predictions generated!!')

In [ ]:
MAX_LEN = 512

generate_prediction(root_path + 'test.data.jsonl', MAX_LEN)

predictions generated!!


-------------------------------------
# This is for testing

In [ ]:
counter = 0
for it, (seq, attn_masks, seg_ids, labels) in enumerate(train_loader):
    if counter > 5:
        break
    counter += 1
    print(it, (seq, attn_masks, seg_ids, labels))

In [ ]:
MAX_LEN = 512


# creating instances of training and development set
test_set = PreprocessDataset(datafile=root_path + 'test.data.jsonl',
                             labelfile=None,
                              maxlen=MAX_LEN)

# creating instances of training and development dataloaders
test_loader = DataLoader(train_set)

with torch.no_grad():
    for seq, attn_masks, seg_ids, labels in test_loader:
        seq, attn_masks, seg_ids, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu), labels.cuda(gpu)
        logits = net(seq, attn_masks, seg_ids)
        probs = torch.sigmoid(logits)
        soft_prob = (probs > 0.5).long()
        decoded_pred = soft_prob.squeeze().item()
        print(decoded_pred)


# prediction = net()

In [ ]:
# Preprocessing attempt
import json
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Define constant
REPLY_ID = 'in_reply_to_status_id_str'
# ID = 'id_str'
TEXT = 'text'
# TIMESTAMP = 'created_at'
MAX_LEN = 512

count = 0

bert_X = []

for index, row in train_df.iterrows():
  source_bool = False
  source_tweet = ''
  reply_tweets = ''

  for tweet in row:
    if tweet == None:
      break
    if tweet[REPLY_ID] == None:
      source_tweet = tweet[TEXT]
    else:
      reply_tweets += tweet[TEXT] + ' '
  
  bert_tokens = bert_tokenizer(source_tweet, reply_tweets, padding=True, truncation=True, max_length=512)
  bert_X.append(bert_tokens)
  # print(bert_tokens)
  # count += 1
  # if count > 2:
  #   break

In [ ]:
# Simplify replies
# Preprocessing attempt
import json
from transformers import AutoTokenizer
import re as re

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet
nltk.download('words')
nltk.download('wordnet')

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
twt_tokenizer = TweetTokenizer()

# Define constant
REPLY_ID = 'in_reply_to_status_id_str'
# ID = 'id_str'
TEXT = 'text'
# TIMESTAMP = 'created_at'
MAX_LEN = 512

count = 0

bert_X = []

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK
words = set([ word.lower() for word in words ]) #lowercase all the words for better matching
words.add('sydney')
words.add('siege')

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def max_match(hashtag):
    hashtag_list = []
    i = 1
    len_hashtag = len(hashtag)

    while i < len_hashtag:
        max_word = ''
        for j in range(i, len_hashtag):
            # If non-alphabet, make it as a single word
            if hashtag[i].isnumeric():
                max_word = hashtag[i]
                break
            temp_word = hashtag[i:j+1]
            # Lemmatise the word first
            lemma_word = lemmatize(temp_word)
            # Then check the lemmaword to corpus
            if lemma_word != temp_word or lemma_word in words:
                # Also check if the tempWord is greater than maxWord
                # Add that to maxWord
                if lemma_word in words and len(temp_word) > len(max_word):
                    max_word = temp_word
        
        # If there is no word detected, tokenise the single word
        if len(max_word) == 0:
            max_word = hashtag[i]
        # Move i with respect to maxWord
        i += len(max_word)
        hashtag_list.append(max_word)

    return hashtag_list

def max_match_rev(hashtag):
        
    hashtag_list = []
    # Append the hashtag since it already tokenised
    i = len(hashtag) - 1
    last_index = len(hashtag) - 1
    
    # Iterate until last_index goes to 0 since it is reversed
    while last_index > 0:
        max_word = ''
        for j in range(last_index, 0, -1):
            # If non-alphabet, make it as a single word
            if hashtag[last_index].isnumeric():
                max_word = hashtag[last_index]
                break
            temp_word = hashtag[j:last_index + 1]
            # Lemmatise the word first
            lemma_word = lemmatize(temp_word)
            # Then check the lemmaword to corpus
            if lemma_word != temp_word or lemma_word in words:
                # Also check if the temp_word is greater than max_word
                # Add that to max_word
                if lemma_word in words and len(temp_word) > len(max_word):
                    max_word = temp_word
        
        # If there is no word detected, tokenise the single word
        if len(max_word) == 0:
            max_word = hashtag[last_index]
        # Move last_index with respect to max_word
        last_index -= len(max_word)
        # Insert the token into index 0
        hashtag_list.insert(0, max_word)
    
    return hashtag_list


row_tweet = list(train_df.iloc[0])
for tweet in row_tweet:
    if tweet == None:
        break
    # tweet_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet[TEXT]).split())
    tweet_text = tweet[TEXT].lower()
    tweet_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', tweet_text, flags=re.MULTILINE)
    tweet_text = re.sub('@[^\s]+', '', tweet_text, flags=re.MULTILINE)
    tweet_text_list = tweet_text.split()
    for word in tweet_text_list:
        if word.startswith("#") and len(word) > 1:
            hashtag_index = tweet_text_list.index(word)
            hashtag_list = max_match_rev(word)
            tweet_text_list.remove(word)
            for tag in reversed(hashtag_list):
                tweet_text_list.insert(hashtag_index, tag)
    separator = ' '
    tweet_text = separator.join(tweet_text_list)
    print(tweet[TEXT].lower())
    print(tweet_text)
    bert_token = bert_tokenizer(tweet_text)
    token_uncleaned = bert_tokenizer(tweet[TEXT].lower())
    # print(bert_tokenizer.decode(token_uncleaned['input_ids']))
    # print(token_uncleaned['input_ids'])
    print(bert_tokenizer.decode(bert_token['input_ids']))
    print(bert_token['input_ids'])



[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
how to respond to the murderous attack on charlie hebdo? every newspaper in the free world should print this. http://t.co/sc2ot63f6j
how to respond to the murderous attack on charlie hebdo? every newspaper in the free world should print this.
[CLS] how to respond to the murderous attack on charlie hebdo? every newspaper in the free world should print this. [SEP]
[101, 2129, 2000, 6869, 2000, 1996, 25303, 2886, 2006, 4918, 2002, 2497, 3527, 1029, 2296, 3780, 1999, 1996, 2489, 2088, 2323, 6140, 2023, 1012, 102]
@heresy_corner @krustyallslopp 
jews label anyone they don't like as anti-semite and campaign until that person/company is finished.
jews label anyone they don't like as anti-semite and campaign until that person/company is finished.
[CLS] jews label anyone t

In [ ]:
# Testing bert
bert_model = BertModel.from_pretrained('bert-base-uncased')

outputs = bert_model(train_set.__getitem__(2)[0],
                     attention_mask=train_set.__getitem__(2)[1],
                     token_type_ids=train_set.__getitem__(2)[2])
hidden_reps = outputs.last_hidden_state
print(hidden_reps.shape)

torch.Size([1, 458, 768])


In [ ]:
import json
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

senc = ['this is a sen', 'this is another', 'and another one']
# print(senc)
senc_tokens = bert_tokenizer(senc)
print(senc_tokens)

{'input_ids': [[101, 2023, 2003, 1037, 12411, 102], [101, 2023, 2003, 2178, 102], [101, 1998, 2178, 2028, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}


In [ ]:
import json
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

REPLY_ID = 'in_reply_to_status_id_str'
ID = 'id_str'

train_id = []

for index, row in train_df.iterrows():
  for tweet in row:
    if tweet[REPLY_ID] == None:
      train_id.append(tweet[ID])
      break
  
with open(root_path + 'train.label.json', 'r') as f:
  train_labels = json.load(f)
  label_df = pd.DataFrame.from_dict(train_labels, orient='index', columns=['label'])
  label_id = le.fit_transform(list(train_labels.values())).tolist()
  label_df = pd.DataFrame(label_id)

print(label_id)
print(le.classes_)
print(label_df)
# print(label_df)

def Diff(li1, li2):
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

# print(Diff(train_id, label_id))

[0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 

In [ ]:
index_test = []

for index, row in train_df.iterrows():
  source_index = 0
  for tweet in row:
    source_index += 1
    if tweet[REPLY_ID] == None:
      index_test.append(source_index)
      break

print(index_test)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
# Inspect hashtags for all the tweets
import pandas as pd

REPLY_ID = 'in_reply_to_status_id_str'
ID = 'id_str'

train_df = pd.read_json(root_path + 'train.data.jsonl', lines=True)

hashtag_list = []

# row = train_df.iloc[14]
# for tweet in row:
#     if tweet == None:
#         break
#     print(tweet)

for index, row in train_df.iterrows():
    for tweet in row:
        if tweet == None:
            break
        text = tweet['text'].split()
        for word in text:
            if word.startswith('#') and len(word) > 1 and word not in hashtag_list:
                hashtag_list.append(word.lower())

# print(hashtag_list)

nltk.download('words')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('reuters')
!unzip /root/nltk_data/corpora/reuters.zip -d /root/nltk_data/corpora

from nltk.corpus import wordnet
from nltk.corpus import brown
from nltk.corpus import reuters


lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK
brown = set(brown.words())
reuters = set(reuters.words())
for w in brown:
    words.add(w)
for w in reuters:
    words.add(w)
words = set([ word.lower() for word in words ]) #lowercase all the words for better matching

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


def tokenize_hashtags(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    
    dictHashtags = dict()
    
    for hashtag in hashtags:
        
        token = []
        
        # Append the hashtag since it already tokenised
        token.append(hashtag[0])
        i = 1
        lenHashtag = len(hashtag)
        
        # Iterate until i reached the total number of words in hashtag
        while i < lenHashtag:
            maxWord = ''
            for j in range(i, lenHashtag):
                # If non-alphabet, make it as a single word
                if hashtag[i].isnumeric():
                    maxWord = hashtag[i]
                    break
                tempWord = hashtag[i:j+1]
                # Lemmatise the word first
                lemmaWord = lemmatize(tempWord)
                # Then check the lemmaword to corpus
                if lemmaWord != tempWord or lemmaWord in words:
                    # Also check if the tempWord is greater than maxWord
                    # Add that to maxWord
                    if lemmaWord in words and len(tempWord) > len(maxWord):
                        maxWord = tempWord
            
            # If there is no word detected, tokenise the single word
            if len(maxWord) == 0:
                maxWord = hashtag[i]
            # Move i with respect to maxWord
            i += len(maxWord)
            token.append(maxWord)
            
        dictHashtags[hashtag] = token
    
    return dictHashtags
    
    ###
    # Your answer ENDS HERE
    ###

#tokenise hashtags with MaxMatch
tokenized_hashtags = tokenize_hashtags(hashtag_list)

#print results
for k, v in sorted(tokenized_hashtags.items()):
    print(k, v)

Streaming output truncated to the last 5000 lines.
  inflating: /root/nltk_data/corpora/reuters/training/95  
  inflating: /root/nltk_data/corpora/reuters/training/9502  
  inflating: /root/nltk_data/corpora/reuters/training/9503  
  inflating: /root/nltk_data/corpora/reuters/training/9505  
  inflating: /root/nltk_data/corpora/reuters/training/9509  
  inflating: /root/nltk_data/corpora/reuters/training/9510  
  inflating: /root/nltk_data/corpora/reuters/training/9511  
  inflating: /root/nltk_data/corpora/reuters/training/9515  
  inflating: /root/nltk_data/corpora/reuters/training/9519  
  inflating: /root/nltk_data/corpora/reuters/training/952  
  inflating: /root/nltk_data/corpora/reuters/training/9521  
  inflating: /root/nltk_data/corpora/reuters/training/9525  
  inflating: /root/nltk_data/corpora/reuters/training/9526  
  inflating: /root/nltk_data/corpora/reuters/training/9527  
  inflating: /root/nltk_data/corpora/reuters/training/953  
  inflating: /root/nltk_data/corpora/r

In [ ]:
words_append = ['toronto', 'sydney', 'hebdo', 'somalia', 'shit', 'fuck', 'wiki',
                'france', 'usa', 'ukraine', 'uganda', ]

for w in reuters:
    if w.lower() == 'charlie':
        print(w)

Charlie
